In [ ]:
!{sys.executable} -m pip install tqdm

In [ ]:

import chromadb
from chromadb.config import Settings
from sentence_transformers import SentenceTransformer
import pandas as pd

# Step 1: Load your CSV file
csv_path = "movies.csv"  # Update if needed
df = pd.read_csv(csv_path)

# Check for "title" field
if "title" not in df.columns:
    raise ValueError("The CSV does not contain a 'title' column.")


C:\Users\John\AppData\Local\Temp\ipykernel_11472\400912964.py:8: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(csv_path)


In [ ]:
# Step 2: Create embeddings using a SentenceTransformer
model = SentenceTransformer("all-MiniLM-L6-v2") # this is a different embedder
texts = df["title"].astype(str).tolist()
embeddings = model.encode(texts, show_progress_bar=True)


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

C:\Users\John\miniconda312\Lib\site-packages\huggingface_hub\file_download.py:144: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\John\.cache\huggingface\hub\models--sentence-transformers--all-MiniLM-L6-v2. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.5k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Batches:   0%|          | 0/1421 [00:00<?, ?it/s]

In [ ]:
# Step 3: Set up Chroma DB and collection

from tqdm import tqdm

# Initialize Chroma client
chroma_client = chromadb.Client()
collection = chroma_client.get_or_create_collection("titles_collection")

# Batch insert (max 5000 per batch)
batch_size = 5000
for i in tqdm(range(0, len(texts), batch_size)):
    batch_texts = texts[i:i+batch_size]
    batch_embeddings = embeddings[i:i+batch_size]
    batch_ids = [f"id_{j}" for j in range(i, i+len(batch_texts))]
    collection.add(
        documents=batch_texts,
        embeddings=batch_embeddings,
        ids=batch_ids
    )

print("All documents inserted in batches.")


100%|██████████████████████████████████████████████████████████████████████████████████| 10/10 [00:42<00:00,  4.25s/it]

All documents inserted in batches.


In [ ]:
# Step 4: Query example
results = collection.query(query_texts=["toy story"], n_results=3)
print("Top Matches:\n", results["documents"])


C:\Users\John\.cache\chroma\onnx_models\all-MiniLM-L6-v2\onnx.tar.gz: 100%|██████| 79.3M/79.3M [01:01<00:00, 1.35MiB/s]


Top Matches:
 [['Toy Story', 'Toy Story 3', 'Toy Story 2']]


In [ ]:
# Step 4: Query example with distances
import time
initial_time = time.time()
results = collection.query(
    query_texts=["toy story"],
    n_results=5,
    include=["documents", "distances"]  # Include relevance scores (lower is better)
)

print("Top Matches:")
for doc, score in zip(results["documents"][0], results["distances"][0]):
    probability = round(1 / (1 + score), 2)  # Convert distance to a pseudo-probability
    print(f"{doc}, confidence: {probability}")
final_time = time.time()
print("Total prediction time", (final_time - initial_time))

Top Matches:
Toy Story, confidence: 1.0
Toy Story 3, confidence: 0.83
Toy Story 2, confidence: 0.82
Toy Story That Time Forgot, confidence: 0.75
The Toy, confidence: 0.66


## Exercise

- Try the very same movie title and compare these results with the ones of google
- Compare the time of prediction of this approach and the one for google
  
